In [82]:
import numpy as np
import pandas as pd
import jsonlines

from sklearn.metrics import f1_score

In [83]:
FILE_TEST_PRED = "scicite/preds/origin/out_test.jsonl"
FILE_TEST_TRUE = "scicite/scicite/data/acl-arc/test.jsonl"

FILE_DEV_PRED = "scicite/preds/origin/out_dev.jsonl"
FILE_DEV_TRUE = "scicite/scicite/data/acl-arc/dev.jsonl"

FILE_TRAIN_PRED = "scicite/preds/origin/out_train.jsonl"
FILE_TRAIN_TRUE = "scicite/scicite/data/acl-arc/train.jsonl"

In [84]:
Labels = ['Background', 'CompareOrContrast', 'Extends', 'Future', 'Motivation', 'Uses']

In [85]:
labels_dict = {}

In [86]:
for i, label in enumerate(Labels):
    labels_dict[label] = i

In [87]:
labels_dict

{'Background': 0,
 'CompareOrContrast': 1,
 'Extends': 2,
 'Future': 3,
 'Motivation': 4,
 'Uses': 5}

In [92]:
def get_f_score(true_file, pred_file):
    # true values
    true = dict()
    with jsonlines.open(true_file) as reader:
        for obj in reader:
            true[obj['citation_id']] = obj['intent']
    
    # true+pred values
    pairs = dict()
    with jsonlines.open(pred_file) as reader:
        for obj in reader:
            pairs[obj['citation_id']] = [true[obj['citation_id']], obj['prediction']]
    
    # encode
    y_true_pred = list(pairs.values())
    y = []
    for pair in y_true_pred:
        y.append([labels_dict[pair[0]], labels_dict[pair[1]]])
    y = np.array(y)
    y_true = y[:, 0]
    y_pred = y[:, 1]
    
    #result
    f_macro = f1_score(y_true, y_pred, average='macro')
    f_micro = f1_score(y_true, y_pred, average='micro')
    
    print(f"F_macro={f_macro}, F_micro={f_micro}, n_samples={len(y_pred)}")
    return f_macro, f_micro

***DEV***

In [89]:
get_f_score(FILE_DEV_TRUE, FILE_DEV_PRED)

F_macro=0.7468576005374051, F_micro=0.7982456140350878, n_samples=114


(0.7468576005374051, 0.7982456140350878)

***TEST***

In [90]:
get_f_score(FILE_TEST_TRUE, FILE_TEST_PRED)

F_macro=0.68523744281723, F_micro=0.7697841726618704, n_samples=139


(0.68523744281723, 0.7697841726618704)

***TRAIN***

In [91]:
get_f_score(FILE_TRAIN_TRUE, FILE_TRAIN_PRED)

F_macro=0.9830034079930069, F_micro=0.9875592417061612, n_samples=1688


(0.9830034079930069, 0.9875592417061612)